<a href="https://colab.research.google.com/github/darthwaydr007/attention-experiments/blob/main/FresUnet_DC_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!gdown https://drive.google.com/uc?id=1JV-_VxTaIcE3hIfmo10I3VPqkibHpuUp


In [ ]:
#!unzip "/content/crop_256.zip" -d "/content/sample_data"

In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 675kB 4.0MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 276kB 42.9MB/s 
     |████████████████████████████████| 829kB 32.6MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=050988f795cb44d7f89fc13688d243ab00b89b5112987fb74a159f8f316bb6ce
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=4ec16d1c4d46ee20f52da61d42ecee026b3dad6cdf1e4fd11176946c248613a8
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully unin

In [ ]:
import torch
from torch import nn
from pytorch_lightning.core.lightning import LightningModule
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision import datasets, transforms
import numpy
import os
import numpy as np
import random
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.datasets.folder import IMG_EXTENSIONS
from PIL import Image
import math
import torch.utils.model_zoo as model_zoo

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU available.' % torch.cuda.device_count())
    print('GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU')
    device = torch.device("cpu")

## Install Data

In [ ]:
import glob 
img_path_A = glob.glob("/content/sample_data/crop_256/train/A/*.png")
img_path_B = glob.glob("/content/sample_data/crop_256/train/B/*.png")
mask_path = glob.glob("/content/sample_data/crop_256/train/label/*.png")

In [ ]:
img_path_A_valid = glob.glob("/content/sample_data/crop_256/val/A/*.png")
img_path_B_valid = glob.glob("/content/sample_data/crop_256/val/B/*.png")
mask_path_valid = glob.glob("/content/sample_data/crop_256/val/label/*.png")

In [ ]:
img_path_A_test = glob.glob("/content/sample_data/crop_256/test/A/*.png")
img_path_B_test = glob.glob("/content/sample_data/crop_256/test/B/*.png")
mask_path_test = glob.glob("/content/sample_data/crop_256/test/label/*.png")

In [ ]:
imageA = Image.open(img_path_A[0])
imageB = Image.open(img_path_B[0])
mask = Image.open(mask_path[0])

In [ ]:
plt.figure()
plt.imshow(imageA) 
plt.show()

In [ ]:
plt.figure()
plt.imshow(imageB) 
plt.show()

In [ ]:
plt.figure()
plt.imshow(mask) 
plt.show()

## Test IOU

In [ ]:
test1 = Image.open("/content/sample_data/crop_256/test/label/test_71_15.png")
test2 = Image.open("/content/sample_data/crop_256/test/label/test_92_8.png")

In [ ]:
trms = transforms.ToTensor()

In [ ]:
testA = trms(test1)
testB = trms(test2)

In [ ]:
plt.figure()
plt.imshow(A[0,:,:]) 
plt.show()

In [ ]:
plt.figure()
plt.imshow(B[0,:,:]) 
plt.show()

In [ ]:
img_and = np.logical_and(A[0,:,:].numpy(), B[0,:,:].numpy()).sum()

In [ ]:
img_or = np.logical_or(A[0,:,:].numpy(), B[0,:,:].numpy()).sum()

In [ ]:
plt.figure()
plt.imshow(img_and) 
plt.show()

In [ ]:
plt.figure()
plt.imshow(img_or) 
plt.show()

In [ ]:
A = testA.unsqueeze(dim = 0)
B = testB.unsqueeze(dim = 0)

In [ ]:
def meanIOU_single(predicted,target):
    iousum = 0
    for i in range(target.shape[0]):
        target_arr = target[i, :, :].clone().detach().cpu().numpy()
        predicted_arr = predicted[i, :, :].clone().detach().cpu().numpy()
        
        intersection = np.logical_and(target_arr, predicted_arr).sum()
        union = np.logical_or(target_arr, predicted_arr).sum()
        
        iou_score = (intersection +1e-10) / (union+1e-10)
        iousum +=iou_score
        
    miou = iousum/target.shape[0]
    return torch.tensor(miou, dtype = torch.float)

In [ ]:
meanIOU_single(B[:,:,:,:],A[:,:,:,:])

## Models

### FresUnet

In [ ]:
from torch.nn.modules.padding import ReplicationPad2d

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1)

In [ ]:
class BasicBlock_ss(pl.LightningModule):

    def __init__(self, inplanes, planes = None, subsamp=1):
        super(BasicBlock_ss, self).__init__()
        if planes == None:
            planes = inplanes * subsamp
        self.conv1 = conv3x3(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.subsamp = subsamp
        self.doit = planes != inplanes
        if self.doit:
            self.couple = nn.Conv2d(inplanes, planes, kernel_size=1)
            self.bnc = nn.BatchNorm2d(planes)

    def forward(self, x):
        if self.doit:
            residual = self.couple(x)
            residual = self.bnc(residual)
        else:
            residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        if self.subsamp > 1:
            out = F.max_pool2d(out, kernel_size=self.subsamp, stride=self.subsamp)
            residual = F.max_pool2d(residual, kernel_size=self.subsamp, stride=self.subsamp)

        out = self.conv2(out)
        out = self.bn2(out)
        
        out += residual
        out = self.relu(out)

        return out


In [ ]:
class BasicBlock_us(pl.LightningModule):

    def __init__(self, inplanes, upsamp=1):
        super(BasicBlock_us, self).__init__()
        planes = int(inplanes / upsamp) # assumes integer result, fix later
        self.conv1 = nn.ConvTranspose2d(inplanes, planes, kernel_size=3, padding=1, stride=upsamp, output_padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.upsamp = upsamp
        self.couple = nn.ConvTranspose2d(inplanes, planes, kernel_size=3, padding=1, stride=upsamp, output_padding=1) 
        self.bnc = nn.BatchNorm2d(planes)

    def forward(self, x):
        residual = self.couple(x)
        residual = self.bnc(residual)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        

        out = self.conv2(out)
        out = self.bn2(out)

        out += residual
        out = self.relu(out)

        return out

In [ ]:
class FresUNet(pl.LightningModule):
    """FresUNet segmentation network."""

    def __init__(self, input_nbr, label_nbr):
        """Init FresUNet fields."""
        super(FresUNet, self).__init__()

        self.input_nbr = input_nbr
        
        cur_depth = input_nbr
        
        base_depth = 16
        
        # Encoding stage 1
        self.encres1_1 = BasicBlock_ss(cur_depth, planes = base_depth)
        cur_depth = base_depth
        d1 = base_depth
        self.encres1_2 = BasicBlock_ss(cur_depth, subsamp=2)
        cur_depth *= 2
        
        # Encoding stage 2
        self.encres2_1 = BasicBlock_ss(cur_depth)
        d2 = cur_depth
        self.encres2_2 = BasicBlock_ss(cur_depth, subsamp=2)
        cur_depth *= 2
        
        # Encoding stage 3
        self.encres3_1 = BasicBlock_ss(cur_depth)
        d3 = cur_depth
        self.encres3_2 = BasicBlock_ss(cur_depth, subsamp=2)
        cur_depth *= 2
        
        # Encoding stage 4
        self.encres4_1 = BasicBlock_ss(cur_depth)
        d4 = cur_depth
        self.encres4_2 = BasicBlock_ss(cur_depth, subsamp=2)
        cur_depth *= 2
        # Decoding stage 4
        self.decres4_1 = BasicBlock_ss(cur_depth)
        self.decres4_2 = BasicBlock_us(cur_depth, upsamp=2)
        cur_depth = int(cur_depth/2)
        
        # Decoding stage 3
        self.decres3_1 = BasicBlock_ss(cur_depth + d4, planes = cur_depth)
        self.decres3_2 = BasicBlock_us(cur_depth, upsamp=2)
        cur_depth = int(cur_depth/2)
        
        # Decoding stage 2
        self.decres2_1 = BasicBlock_ss(cur_depth + d3, planes = cur_depth)
        self.decres2_2 = BasicBlock_us(cur_depth, upsamp=2)
        cur_depth = int(cur_depth/2)
        
        # Decoding stage 1
        self.decres1_1 = BasicBlock_ss(cur_depth + d2, planes = cur_depth)
        self.decres1_2 = BasicBlock_us(cur_depth, upsamp=2)
        cur_depth = int(cur_depth/2)
        
        # Output
        self.coupling = nn.Conv2d(cur_depth + d1, label_nbr, kernel_size=1)
        self.sm = nn.LogSoftmax(dim=1)
        
    def forward(self, x1, x2):

        x = torch.cat((x1, x2), 1)
        
#         pad5 = ReplicationPad2d((0, x53.size(3) - x5d.size(3), 0, x53.size(2) - x5d.size(2)))
        
        s1_1 = x.size()
        x1 = self.encres1_1(x)
        x = self.encres1_2(x1)
        
        s2_1 = x.size()
        x2 = self.encres2_1(x)
        x = self.encres2_2(x2)
        
        s3_1 = x.size()
        x3 = self.encres3_1(x)
        x = self.encres3_2(x3)

        s4_1 = x.size()
        x4 = self.encres4_1(x)
        x = self.encres4_2(x4)
        
        x = self.decres4_1(x)
        x = self.decres4_2(x)
        s4_2 = x.size()
        pad4 = ReplicationPad2d((0, s4_1[3] - s4_2[3], 0, s4_1[2] - s4_2[2]))
        x = pad4(x)
        
        # x = self.decres3_1(x)
        x = self.decres3_1(torch.cat((x, x4), 1))
        x = self.decres3_2(x)
        s3_2 = x.size()
        pad3 = ReplicationPad2d((0, s3_1[3] - s3_2[3], 0, s3_1[2] - s3_2[2]))
        x = pad3(x)
        
        x = self.decres2_1(torch.cat((x, x3), 1))
        x = self.decres2_2(x)
        s2_2 = x.size()
        pad2 = ReplicationPad2d((0, s2_1[3] - s2_2[3], 0, s2_1[2] - s2_2[2]))
        x = pad2(x)
        
        x = self.decres1_1(torch.cat((x, x2), 1))
        x = self.decres1_2(x)
        s1_2 = x.size()
        pad1 = ReplicationPad2d((0, s1_1[3] - s1_2[3], 0, s1_1[2] - s1_2[2]))
        x = pad1(x)
        
        x = self.coupling(torch.cat((x, x1), 1))
        #x = self.sm(x)
        
        return x

### Unet

### SiameseUnet

## Dataset and DataLoader

In [ ]:
BATCH_SIZE = 32

In [ ]:
class LEVIRDataset(Dataset):
  def __init__(self , img_path_A, img_path_B, mask_path):
    self.img_path_A = img_path_A
    self.img_path_B = img_path_B
    self.mask_path = mask_path
    self.length = len(self.img_path_A)
    self.transforms = transforms.Compose([
                                          transforms.ToTensor()])

  def __len__(self):
    return self.length

  def __getitem__(self , index):   
    return {
        'imgA' : self.transforms(Image.open(self.img_path_A[index])) , 
        'imgB' : self.transforms(Image.open(self.img_path_B[index])) ,
        'mask' : self.transforms(Image.open(self.mask_path[index])) , 
    }

In [ ]:
class LEVIRDateModule(pl.LightningDataModule):

  def __init__(self, batch_size):
    super().__init__()
    self.batch_size = batch_size

  def setup(self):
    self.train_dataset = LEVIRDataset(img_path_A ,img_path_B, mask_path)
    self.valid_dataset = LEVIRDataset(img_path_A_valid ,img_path_B_valid, mask_path_valid)
    self.test_dataset = LEVIRDataset(img_path_A_test ,img_path_B_test, mask_path_test)

  def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

  def val_dataloader(self):
      return DataLoader(self.valid_dataset, batch_size=self.batch_size)

  def test_dataloader(self):
      return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [ ]:
data_module = LEVIRDateModule(batch_size = 16)
data_module.setup()

## Optimiiser

In [ ]:
from pytorch_lightning import Trainer, seed_everything
seed_everything(0)

In [ ]:
sigmoid = nn.Sigmoid()
criterion = nn.CrossEntropyLoss()

In [ ]:
from pytorch_lightning.metrics import Metric

class IOU(Metric):
    def __init__(self, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)

        self.add_state("iou", default=torch.tensor(0, dtype = torch.float), dist_reduce_fx="sum")
        self.add_state("count", default=torch.tensor(0 , dtype = torch.float), dist_reduce_fx="sum")

    def update(self, preds, target):
        #preds, target = self._input_format(preds, target)
        #assert preds.shape == target.shape
        self.iou += meanIOU_single(preds,target)
        self.count += 1

    def compute(self):
        return self.iou / self.count

In [ ]:
class Model(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = FresUNet(3*2, 2)
    self.train_iou = IOU()
    self.valid_iou = IOU()

  
  def forward(self, imgA , imgB):
    out = self.model(imgA , imgB)
    return out

  #def cross_entropy_loss(self, logits, labels):
  #  return nn.CrossEntropyLoss(logits, labels)

  def configure_optimizers(self):
    optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.5, 0.999), weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    return optimizer , scheduler

  #def training_step_end(self, batch_parts):
  #  gpu_0_prediction = batch_parts[0]['pred']
  #  gpu_1_prediction = batch_parts[1]['pred']

  def training_step(self, batch, batch_idx):
    imgA = batch['imgA']
    imgB = batch['imgB']
    mask1 = batch['mask']
    label = mask1[:,0,:,:]
    label[label>0] = 1
    label = label.long()

    output = self.model(imgA , imgB)
    loss = criterion(output , label)

    labelx = mask1[:,0,:,:]
    preds = sigmoid(output)
    preds[preds>=0.5] = 1
    preds[preds<0.5] = 0

    #iou = meanIOU_single(labelx, preds[:,1,:,:])
    self.train_iou(preds[:,1,:,:].long(), labelx.long())
    self.log('train_iou', self.train_iou, on_step=True, on_epoch=True , prog_bar=True, logger=True)
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    imgA = batch['imgA']
    imgB = batch['imgB']
    mask1 = batch['mask']
    label = mask1[:,0,:,:]
    label[label>0] = 1
    label = label.long()

    output = self.model(imgA , imgB)
    loss = criterion(output , label)

    labelx = mask1[:,0,:,:]
    preds = sigmoid(output)
    preds[preds>=0.5] = 1
    preds[preds<0.5] = 0


    #iou = meanIOU_single(labelx, preds[:,1,:,:])
    self.valid_iou(preds[:,1,:,:], labelx)
    self.log('valid_iou', self.valid_iou, on_step=True, on_epoch=True , prog_bar=True, logger=True)
    self.log('valid_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)


In [ ]:
model = Model()

In [ ]:
trainer = pl.Trainer(gpus=1 , precision = 16)

In [ ]:
trainer.fit(model, data_module)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/